In [1]:
import tensorflow as tf
import numpy as np
from scipy.io import wavfile

2023-04-06 09:11:28.002282: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-06 09:11:28.353835: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-06 09:11:28.355610: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 09:11:29.598500: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
interpreter = tf.lite.Interpreter('./models/classifier/00000001/model.tflite')

In [3]:
input_details = interpreter.get_input_details()
waveform_input_index = input_details[0]['index']
output_details = interpreter.get_output_details()
scores_output_index = output_details[0]['index']

In [4]:
samplerate, waveform = wavfile.read('./int.wav')

In [18]:
import scipy.signal as sps

# Your new sampling rate
new_rate = 16000

# Resample data
number_of_samples = round(len(waveform) * float(new_rate) / samplerate)
data = sps.resample(waveform, number_of_samples)

In [5]:
import json
import requests
import zipfile

labels_file = zipfile.ZipFile('./models/classifier/00000001/model.tflite').open('yamnet_label_list.txt')
labels = [l.decode('utf-8').strip() for l in labels_file.readlines()]

In [31]:
import pandas as pd

In [81]:
file = json.load(open("ontology.json", "r"))

In [144]:
result = []
def get_tree(tree, path=[]):
    for child in tree:
        path.append(child["name"])
        if child.get("children"):
            get_tree(child.get("children"), path)
        else:
            result.append(path.copy())
            print(path)
            path.pop()
    if path:
        path.pop()

In [145]:
get_tree(file["children"])

['Natural sounds', 'Wind', 'Howl (wind)']
['Natural sounds', 'Wind', 'Rustling leaves']
['Natural sounds', 'Wind', 'Wind noise (microphone)']
['Natural sounds', 'Thunderstorm', 'Thunder']
['Natural sounds', 'Water', 'Rain', 'Raindrop']
['Natural sounds', 'Water', 'Rain', 'Rain on surface']
['Natural sounds', 'Water', 'Stream']
['Natural sounds', 'Water', 'Waterfall']
['Natural sounds', 'Water', 'Ocean', 'Waves, surf']
['Natural sounds', 'Water', 'Steam', 'Hiss']
['Natural sounds', 'Water', 'Gurgling']
['Natural sounds', 'Fire', 'Crackle']
['Natural sounds', 'Fire', 'Wildfire']
['Channel, environment and background', 'Acoustic environment', 'Inside, small room']
['Channel, environment and background', 'Acoustic environment', 'Inside, large room or hall']
['Channel, environment and background', 'Acoustic environment', 'Inside, public space']
['Channel, environment and background', 'Acoustic environment', 'Outside, urban or manmade']
['Channel, environment and background', 'Acoustic envir

In [147]:
pd.DataFrame(result).to_csv("classes_tree.csv", index=False)

In [21]:
data.astype(np.float32).max()
audio = data/np.max(np.abs(data),axis=0)

In [25]:
waveform.max()

32767

In [23]:
data

array([ 0.56927568, -0.56864424,  0.56801136, ..., -0.57116144,
        0.57053428, -0.5699057 ])

In [28]:
audio

array([ 1.70963020e-05, -1.70773386e-05,  1.70583321e-05, ...,
       -1.71529344e-05,  1.71340999e-05, -1.71152224e-05])

In [29]:
for el in range(0, len(audio), 15600):
    signal = audio[el:el+15600].tolist()
    result = requests.post("http://localhost:8501/v1/models/classifier:predict", data=json.dumps({"instances": signal}))
    if result:
        print(labels[np.array(json.loads(result.content)["predictions"]).argmax()])

Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Music
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Wild animals
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Animal
Typing
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Silence
Speech
Speech
Speech
Speech
Speech
Bird
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Speech
Sp

In [23]:
interpreter.resize_tensor_input(waveform_input_index, [waveform.size], strict=True)
interpreter.allocate_tensors()
interpreter.set_tensor(waveform_input_index, waveform)
interpreter.invoke()
scores = interpreter.get_tensor(scores_output_index)
print(scores.shape)  # Should print (1, 521)

(1, 521)


In [19]:
# top_class_index = scores.argmax()
labels_file = zipfile.ZipFile('./models/classifier/00000001/model.tflite').open('yamnet_label_list.txt')
labels = [l.decode('utf-8').strip() for l in labels_file.readlines()]
print(len(labels))  # Should print 521
print(labels[462])


521
Whack, thwack
